# Objetivo de este notebook

Con este notebook se exhibiran algunos conceptos, algoritmos o funciones para establecer un modelo de recomendación de películas

# Requisitos

In [1]:
#Para correr las funciones de la librería libmf de python de este notebook, 
#debes clonar el repo de https://github.com/PorkShoulderHolder/python-libmf.git
#subir este notebook dentro de la carpeta /libmf/tests/

#Para correr las funciones de libmf de este notebook ejecutados con 
#Shell Commands ("!") y Shell-Related Magic Commands ("%")
#debes clonar el repo de https://github.com/cjlin1/libmf.git

#El clonar estos archivos es necesario porque contienen funciones programadas o
#ejemplos construidos para exhibir el uso de estos paquetes

#Existen bases que requieres tener la carpeta donde corras el código
#a saber: movies_title_fix.csv dat_muestra_nflix.csv

#si queres correr este notebook a través de docker con la imagen de
#jupyer numerical, debes instalar los paquetes libmf y colorama,
#ejecutando los comandos de abajo, reiniciando el kernel (luego
#puedes comentar con "#" los "pip install" para que no se vuelvan a ejecutar)

In [2]:
#pip install libmf

In [3]:
#pip install colorama

In [4]:
import sys
import numpy as np
import types
import scipy
from scipy.sparse import random
from scipy import stats
import pandas as pd
import copy
import random
from functools import partial
from colorama import init

import libmf
from libmf import mf

Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so


# Funciones auxiliares
Para explicar y ejemplificar el uso de libmf

In [5]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def constante(x,useless):
    """
    Devuelve la constante x
    
    params: x       float constante que se quiere devolver
            useless float cualquier valor que se quiera pasar
    return: x       Devuelve la misma constante
    """
    return x

In [6]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def perform(fun, *args):
    """
    Evalúa una función con los argumentos indicados
    
    params: fun         Una función
            *args       Parámetros que utiliza la función "fun"
                        (se pueden meter cuantos parámetros tenga 
                        dicha funcion)
    return: fun(*args)  Valor de fun con parámetros *args
    """
    return fun(*args)

In [7]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def nan_or_value(x, f, *args):
    """
    Devuelve el valor de una función evaluada con sus parámetros
    *args acorde a la variable indicadora x
    
    params: x         boolean, si es 1 tómese valor de la función
                      si no será nan o 0 según especifique nan_zero
            fun       Una función
            *args     Parámetros que utiliza la función "fun"
                      (se pueden meter cuantos parámetros tenga 
                      dicha funcion)
            nan_zero  Si no se especifica nan devuelve 0
    return: fun(*args)  Valor de fun con parámetros *args
    """
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [8]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)

#Con esta vectorización se podrá aplicar a todos los elementos
#de un np.array la función nan_or_value
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

In [9]:
#Función para generar matrices ralas (sólo se usa para un ejemplo)
#Por su forma de construcción esta función es bastante ineficiente para
#ejemplos grandes
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán nan cumpliendo que la proporción de éstos es de 
    1-prop_elem_difer_nan, donde los numeros diferentes de nan serán aleatorios
    generados a partir de una funcion aletorea que se envíe como parámetro
    (y sus argumentos)
    
    params: n_ren               int numero de renglones que tiene la matriz
            m_col               int numero de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
            func_rand           Función que genera un sólo no. aleatorio de
                                tipo float (no arreglos)
            *args_fun_rand      Argumentos necesarios para ejecutar la
                                la funcoión fun_rand
            nan_zero            Si no se especifica nan devuelve 0
    return: nan_or_value_vec    np.ndarray que contiene matriz rala
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    #se juntan 2 arreglos, uno de ceros y otro de unos, para después
    #reacomodarlos al azar
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    #Los unos se intercambias por numeros aleatorios y los ceros
    #por nan (o se dejan como cero)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

In [10]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def compactar_matriz_rala(M):
    """
    Pasa una matriz rala a su expresión compacta
    
    params: M  np.ndarray que representa matriz rala
    return: MC np.ndarray matriz en expresión compacta
               (no se reportarán los elementos con valor
               nan)
    """
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

In [11]:
#Función para generar matrices ralas en su expresión compacta
#Utilizada para varios ejemplos
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán distintos de nan. Éstos elementos distintos de nan serán
    numero aleatorios entre 1 y 5 (uniforme discreta). Se dice compacta porque
    es una matriz solo señala las coordenadas o entredas de la matriz que 
    son diferentes a nan
    
    params: n_ren               int numero de renglones que tiene la matriz
            m_col               int numero de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
    return: M  np.ndarray que contiene matriz rala "compacta"
               (1era columna representa el índice de renglones,
               2da columna representa el índice de columnas,
               3era columna representa el valor del elemento
               o calificación en el contexto de netflix)
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    M=np.concatenate([renglones, columnas, aleatorios])
    M=M.reshape(3,k).T
    return M

In [12]:
#Función auxiliar al procesamiento de datos para la aplicación de un
#modelo de recomendación
def modelo_base_ref(MC):
    """
    Obtiene promedios por índices de renglón y columnas, así como del total
    (o calificaciones en el contexto de películas netflix)
    
    params: MC np.ndarray (con 3 columnasm, la 1era se toma como la
               de índices de renglon, la 2da como la de índices de columna
               y la 3era como el valor de los elementos o calificaciones en
               el contexto de netflix)
    return: count_by  pd.DataFrame de 3 columnas que tiene sólo los índices de
                      renglones o columnas que tienen al menos n elementos
    """
    df=pd.DataFrame(MC,columns=['row','col','score'])
    #Media total
    c=df['score'].mean(axis=0)

    #media por columna
    mean_by_col=df[['col','score']].groupby(['col']).mean()
    mean_by_col.rename(columns={'score':'mean_score_by_col'}, inplace=True)
    #media por renglon
    mean_by_row=df[['row','score']].groupby(['row']).mean()
    mean_by_row.rename(columns={'score':'mean_score_by_row'}, inplace=True)

    #se unen todas los resultados obtenidos, incluyendo la columna de
    #índice de columnas y renglones
    df=df.set_index('col').join(mean_by_col,how='left')
    df['col']=df.index
    df=df.set_index('row').join(mean_by_row,how='left')
    df['row']=df.index
    df=df[['row','col','mean_score_by_row','mean_score_by_col']]
    df['total_score_mean']=c
    
    return df

In [13]:
#Función auxiliar al procesamiento de datos para la aplicación de un
#modelo de recomendación
def filtrar(MC,ren_o_col,n):
    """
    Filtra los índices de renglon o columna que tengan más de n elementos
    (o calificaciones en el contexto de películas netflix)
    
    params: MC        np.ndarray (con 3 columnas, de nombre indistinto)
            ren_o_col string igual a 'row' o 'col'
            n         int no. elementos mínimos requeridos
    
    return: count_by  np.ndarray de 3 columnas que tiene sólo los índices de
                      renglones o columnas que tienen al menos n elementos
    """
    MC_df=pd.DataFrame(MC,columns=['row','col','score'])
    
    #este pequeño bloque calcula el total de elementos por columna o renglón
    count_by=MC_df[[ren_o_col,'score']].groupby([ren_o_col]).count()
    count_by.rename(columns={'score':'tot_counted'}, inplace=True)
    count_by[ren_o_col]=count_by.index
    count_by=count_by[[ren_o_col,'tot_counted']]
    
    #en este bloque se filtran los elementos de la matriz cuyo conteo por
    #columna o renglón tienen más de n elementos
    count_by=count_by[count_by.apply(lambda x: x['tot_counted'] >= n, axis=1)]
    count_by=MC_df.set_index(ren_o_col).join(count_by.set_index(ren_o_col),how='inner')
    count_by[ren_o_col]=count_by.index
    count_by=count_by[['row','col','score']]
    
    return count_by.to_numpy()

In [14]:
#Función auxiliar para obtención de muestras aletorias
def muestra_val_ent(df,prop):
    """
    Separa al azar una base dada en 2 bases acorde una proporción de elemtos
    dada
    
    params: df    pd.DataFrame (con 3 columnas=['usuario_id','peli_id',
                  'calif_x'])
            prop  float (0 < prop < 1) proporción de id_usuarios y
                  proporción de id_peliculas que se seleccionarán
    
    return: data_test      pd.DataFrame que tiene base que se usará
                           para prueba (por construcción el numero de
                           elementos de esta base no necesariamente coincide
                           con prop*total de elementos de df)
            data_training  pd.DataFrame que tiene base que se usará
                           para entrenamiento
    """
    #Base de todos los usuarios
    usuarios=pd.DataFrame(df['usuario_id'].unique(), columns=['usuario_id'])
    #Muestreo de usuarios
    valida_usuarios=usuarios.sample(frac=prop)  

    #Base de todas las películas
    peliculas=pd.DataFrame(df['peli_id'].unique(), columns=['peli_id'])
    #Muestreo de películas
    valida_pelis=peliculas.sample(frac=prop)
    
    #df_valida tiene las columnas usuario_id, peli_id y calif, pero
    #sólo incluye los usuarios muestreados en valida_usuarios
    df_valida=pd.merge(df,valida_usuarios, on ='usuario_id', how='inner')
    #data_valida filtra de la base df_valida los pelis_id muestreas
    #en valida_pelis
    data_test=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
    left=pd.merge(df, data_test, how='left',on=['usuario_id', 'peli_id'])
    data_training=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]
    return data_test, data_training

In [15]:
def imprime_atributos_MF(Obj):
    """
    Imprime los parámetros utilizados para factorización de la matriz contenida
    en Obj, mismos que son atributos del atributo _options
    
    params: Obj  libmf.mf.MF objeto de la librería libmf-python que contiene
                 la información requerida para hacer la factorización deseada
    
    return: None No regresa nada, sólo imprime los parámetros utilizados para
                 hacer la factorización
    """
    if type(Obj) is not libmf.mf.MF:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Atributos del modelo:')
    print('fun={0:d}'.format(Obj._options.fun))
    print('k={0:d}'.format(Obj._options.k))
    print('nr_threads={0:d}'.format(Obj._options.nr_threads))
    print('nr_bins={0:d}'.format(Obj._options.nr_bins))
    print('nr_iters={0:d}'.format(Obj._options.nr_iters))
    print('lambda_p1={0:0.4f}'.format(Obj._options.lambda_p1))
    print('lambda_p2={0:0.4f}'.format(Obj._options.lambda_p2))
    print('lambda_q1={0:0.4f}'.format(Obj._options.lambda_q1))
    print('lambda_q2={0:0.4f}'.format(Obj._options.lambda_q2))
    print('eta={0:0.4f}'.format(Obj._options.eta))
    print('do_nmf=',Obj._options.do_nmf)
    print('quiet=',Obj._options.quiet)
    print('copy_data=',Obj._options.copy_data)
    return None

In [16]:
# class MFModel(ctypes.Structure):
#     _fields_ = [("fun", ctypes.c_int),
#                 ("m", ctypes.c_int),
#                 ("n", ctypes.c_int),
#                 ("k", ctypes.c_int),
#                 ("b", ctypes.c_float),
#                 ("P", c_float_p),
#                 ("Q", c_float_p)]

In [17]:
#Elegiremos un formato de general para visualizar las matrices exhibidas como ejemplo
np.set_printoptions(sign=' ', precision=3)
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})

# Matrices ralas (sparse)

In [18]:
#En este proyecto se exhibe el uso de las funciones de la librería libmf
#relacionadas a la factorización de matrices. El proyecto se enfoca al uso
#de estos algoritmos/métodos para los sistemas de recomendación de películas
#cuyo objetivo será obtener recomendaciones de películas para usuarios
#contenidos dentro de una base de datos proporcionada por netflix

#Si bien la base tiene datos como peli_id, usuario_id, calif, fecha, el
#algoritmo solo trabajará con los datos de calificaciones. Si bien existen
#diversos usuarios y películas, son pocas películas de las que se tienen
#calificaciones (escala del 1 al 5) proporcionadas por usuarios, por lo que
#la representación de estas calificaciones a través de una matriz (donde
#los usuarios serán los índices de los rengloens, las películas los índices
#de columnas y las calificaciones los valores de las entradas de la matriz),
#contendrá muchos huecos o calificaciones inexistentes,
#es decir, es una matriz rala (llena de ceros o valores no proporcionados)
#No obstante lo anterior, debido al numero de usuarios y películas
#existen muchos datos, y para no guardar todos los valores de cero (o NA)
#se opta por expresar y guardar dicha matriz en una forma compacta,
#la cual contiene por cada calificación el usuario que la generó y la película
#a la que corrresponde (es decir, es una matriz con 3 columnas)

In [19]:
#Crear matriz rala de tamaño 20x5, donde el 25% de sus elementos
#será distinto de nan, señalando la distribución de probabilidad
#que tendrán dichos elementos
M=crear_matriz_rala(20,5,0.25,stats.randint.rvs,*[1,6])
#Puedes cambiar la distribución aleatoria, prueba con las
#distribuciones existentes en scipy.stats o la función constante
#R=crear_matriz_rala(20,5,0.5,scipy.stats.uniform.rvs,*[1,4])
#R=crear_matriz_rala(20,5,0.2,stats.binom.rvs,*[5,0.2])
#R=crear_matriz_rala(20,5,0.5,constante,*[1,0])
print('Matriz M:\n',M)
print('Tamaño M:\n',M.shape)
print('Numero de elementos diferentes de nan:\n',M[np.isnan(M)==False].size)

Matriz M:
 [[  5.  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan   5.   4.]
 [  3.  nan  nan   1.   2.]
 [ nan  nan  nan   1.  nan]
 [ nan  nan   1.  nan  nan]
 [  1.   1.  nan  nan  nan]
 [ nan  nan   3.  nan  nan]
 [ nan   2.  nan   3.   4.]
 [ nan  nan  nan  nan  nan]
 [ nan   3.  nan  nan   1.]
 [ nan  nan  nan  nan  nan]
 [ nan   5.  nan  nan  nan]
 [ nan  nan  nan   1.  nan]
 [ nan  nan  nan   3.  nan]
 [ nan   5.  nan  nan  nan]
 [ nan  nan   2.   3.  nan]
 [ nan   1.  nan   2.  nan]
 [ nan   2.  nan  nan  nan]]
Tamaño M:
 (20, 5)
Numero de elementos diferentes de nan:
 25


In [20]:
#Expresión compacta de la matriz rala (con 3 columnas: col1,col2,col3),
#que señala el índice de renglón (col1) y columna (col2) de la matriz R,
#correspondiente a los elementos distitnos de nan, así como el valor
#de dicho elemento (col3)
MC=compactar_matriz_rala(M)
print('Expresión compacta de M (MC):\n',MC)
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC):
 [[  0.   0.   5.]
 [  3.   3.   5.]
 [  3.   4.   4.]
 [  4.   0.   3.]
 [  4.   3.   1.]
 [  4.   4.   2.]
 [  5.   3.   1.]
 [  6.   2.   1.]
 [  7.   0.   1.]
 [  7.   1.   1.]
 [  8.   2.   3.]
 [  9.   1.   2.]
 [  9.   3.   3.]
 [  9.   4.   4.]
 [ 11.   1.   3.]
 [ 11.   4.   1.]
 [ 13.   1.   5.]
 [ 14.   3.   1.]
 [ 15.   3.   3.]
 [ 16.   1.   5.]
 [ 17.   2.   2.]
 [ 17.   3.   3.]
 [ 18.   1.   1.]
 [ 18.   3.   2.]
 [ 19.   1.   2.]]
Tamaño MC:
 (25, 3)


In [21]:
#El presente trabajo plantea trabajar con bases de películas, donde
#los ínidices i de renglones representan usuarios, los ínidces j de columnas
#las películas, y el valor de elemento de la matriz la calificación
#asignada a la película j por el usuari i
#Por lo anterior, para crear matrices compactas ralas usaremos funciones propias
#donde un usuario no califique más de una vez una misma película (lo cual 
#sucediería si se genera la matriz de calificaciones con la función de la
#paquetería libmf). La calificación será un no. aleatorio entre 1 y 5
#generada con una dist. uniforme discreta
MC=crear_matriz_rala_compacta(20,5,0.25)
print('Expresión compacta de M (MC) ordenada por renglon:\n',MC[np.argsort(MC[:,0])])
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC) ordenada por renglon:
 [[ 0  3  5]
 [ 1  2  1]
 [ 1  0  3]
 [ 2  1  5]
 [ 3  1  4]
 [ 4  3  4]
 [ 6  1  3]
 [ 6  4  1]
 [ 7  3  5]
 [ 7  4  1]
 [ 7  2  3]
 [ 8  4  5]
 [ 9  2  3]
 [11  1  1]
 [11  0  3]
 [12  1  5]
 [14  1  2]
 [15  3  4]
 [15  1  4]
 [15  4  1]
 [16  1  5]
 [16  0  5]
 [17  2  2]
 [17  1  3]
 [18  0  4]]
Tamaño MC:
 (25, 3)


In [22]:
#Aquí una forma de gardar la matriz compacta en un txt
#que podría ser utilizado directamente con las funciones
#de libmf a través de Shell Commands ("!")
np.savetxt('MC.txt', MC, delimiter=' ', fmt="%.0f")
#Se imprime lo contenido en el .txt
f = open('MC.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

17 1 3
1 2 1
6 1 3
15 3 4
6 4 1
7 3 5
3 1 4
7 4 1
7 2 3
14 1 2
0 3 5
17 2 2
16 1 5
18 0 4
4 3 4
11 0 3
15 1 4
12 1 5
9 2 3
16 0 5
2 1 5
8 4 5
11 1 1
1 0 3
15 4 1



# Procesos útiles para construcción de modelo
### Calificaciones promedio por usuario y/o película, y filtrado

In [23]:
#Para plantear un "modelo base de referencia", se calculan los medias
#por renglones (mean_ren) y por columnas (mean_col), así como 
#una media total (mean_tot), luego se sustituyen los valores de 
#la matriz (x_ij) por: mean_ren + mean_col - mean_tot
#La función modelo_base_ref regresa los promedios por renglon,
#columna y total. Con estos promedios se puede construir un modelo
#para heterogenizar el uso de la escala (calificaciones)

promedios=modelo_base_ref(MC)
display(promedios.head())
#podemos convertir a numpy array para realizar operaciones con
#variables de este tipo

#Recuerda pasa a np.array para hacer operaciones
promedios=promedios.to_numpy()
#el modelo base de ref quedaría así:
print('\n El modelo base de referencia es mean_ren + mean_col - mean_tot:') 
promedios[:,0]+promedios[:,1]-promedios[:,2]

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
0,0,3,5.0,4.500000,3.28
1,1,0,2.0,3.750000,3.28
1,1,2,2.0,2.250000,3.28
2,2,1,5.0,3.555556,3.28
3,3,1,4.0,3.555556,3.28



 El modelo base de referencia es mean_ren + mean_col - mean_tot:


array([-2. , -1. ,  1. , -2. ,  0. ,  3. ,  5. ,  8. ,  6. ,  7. ,  8. ,
        7. ,  8. ,  9. , 10. ,  8. , 13. , 13. , 15. , 16. , 11. , 12. ,
       15.5, 16.5, 14. ])

In [24]:
#En el camino para construir modelos adecuados, puede ser de interés
#o utilidad identificar los usuarios que tiene más películas calificadas
#o las películas con más calificaciones, más precisamente, que tengan
#más de "n" calificaciones.

#Usuarios que calificaron más de 2 películas
display(filtrar(MC,'row',2))
#Películas que fueron calificadas por más de 2 usuarios
display(filtrar(MC,'col',2))

array([[ 1,  2,  1],
       [ 1,  0,  3],
       [ 6,  1,  3],
       [ 6,  4,  1],
       [ 7,  3,  5],
       [ 7,  4,  1],
       [ 7,  2,  3],
       [11,  0,  3],
       [11,  1,  1],
       [15,  3,  4],
       [15,  1,  4],
       [15,  4,  1],
       [16,  1,  5],
       [16,  0,  5],
       [17,  1,  3],
       [17,  2,  2]])

array([[18,  0,  4],
       [11,  0,  3],
       [16,  0,  5],
       [ 1,  0,  3],
       [17,  1,  3],
       [ 6,  1,  3],
       [ 3,  1,  4],
       [14,  1,  2],
       [16,  1,  5],
       [15,  1,  4],
       [12,  1,  5],
       [ 2,  1,  5],
       [11,  1,  1],
       [ 1,  2,  1],
       [ 7,  2,  3],
       [17,  2,  2],
       [ 9,  2,  3],
       [15,  3,  4],
       [ 7,  3,  5],
       [ 0,  3,  5],
       [ 4,  3,  4],
       [ 6,  4,  1],
       [ 7,  4,  1],
       [ 8,  4,  5],
       [15,  4,  1]])

# Librería libmf python

In [25]:
# Python-libmf contiene de fondo el mismo código que el de la librería libmf, 
# aunque tiene programadas clases con atributos y métodos para poder ejecutar 
# los métodos de libmf. A continuación se señalan las principales 
# clases, atributos y métodos

# Clase: MF
# Atributos: .model
#            ._options 
#            .i
#            .j
#
# Métodos: .fit -> < Objetivo: factorize the i x j data matrix X into (j, k) (k, i) 
#                  <           sized matrices stored in MF.model
#                  < Parámetro X: (n, 3) shaped numpy array [known index and values 
#                  <              of the data matrix]
#
#          .predict -> < Objetivo: assuming we have already run the fit method, predict
#                      <           the values at certain indices of the data matrix
#                      < Parámetro X: (n, 2) shaped numpy array
#                      < Regresa: numpy array of length n
#
#          .mf_cross_validation -> < Objetivo: Realizar cross-validation
#                                  < Parámetro X: (n, 3)
#                                  < Parámetro folds: number of train / test splits
#                                  < Regresa: score across all folds 
#
#          .mf_train_test -> < Objetivo: ??????????????????????????
#                            < Parámetro X: matriz de entrenamiento
#                            < Parámetro V: matriz de validación
#
#          .q_factors -> < Objetivo: Obtener la matriz Q
#
#          .p_factors -> < Objetivo:  Obtener la matriz P


# Clase: model
# Atributos: .fun
#            .m 
#            .n
#            .k
#            .b
#            .P
#            .Q
# Métodos: NA


# Clase: _options
# Atributos: .fun
#            .k
#            .nr_threads
#            .nr_bins
#            .nr_iters
#            .lambda_p1
#            .lambda_p2
#            .lambda_q1
#            .lambda_q2
#            .eta
#            .do_nmf
#            .quiet
#            .copy_data
# Métodos: NA


In [26]:
#A continuación un comparativo de los parámetros usados por python-libmf 
#y libmf, así como sus valores predeterminados

|Parámetro:|Nombre python-libmf|Valor por default python-libmf|Nombre libmf|Valor por default libmf|
|:--:|:--:|:--:|:--:|:--:|
|Función de pérdida|fun|0|-f|0|
|Numero de factores latentes|k|8|-k|8|
|Numero de threads|nr_threads|12|-s|12|
|Numero de bins|nr_bins|26|-n|adjusted by LIBMF for speed|
|Numero de iteraciones|nr_iters|20|-t|20|
|Parámetro de regularización L1 para P y Q<sup>(1)</sup>|lambda_p1|0.04|-l1|0|
|Parámetro de regularización L2 para P y Q<sup>(1)</sup>|lambda_p2|0|-l2|0.1|
|Parámetro de regularización L1 para P y Q<sup>(1)</sup>|lambda_q1|0.04|-l1|0|
|Parámetro de regularización L2 para P y Q<sup>(1)</sup>|lambda_q2|0|-l1|0.1|
|Tasa inicial de aprendizaje|eta|0.1|-r|0.1|
|Realizar factorización matricial no negativa<sup>(3)</sup>|do_nmf|False|--nmf|False|
|Sin outputs<sup>(3)</sup>|quite|False|--quite|False|
|Copiar datos|copy_data|True|NA|NA|
|Ruta conjunto validación|NA|NA|-p|???|
|Numero de fold para validación cruzada|NA|NA|-v|1|
|Coeficiente de pérdida de entradas negativas|NA|NA|-a|1|
|Valor de entradas negativas<sup>(2)</sup>|NA|NA|-c|0.0001|
|Realizar entrenamiento a nivel de disco<sup>(3)</sup>|NA|NA|--disk|False|

<sup>(1)</sup> En libmf "l1" y "l2" se pueden definir con 1 o 2 parámetros, si se define uno se entiende que aplica tanto para P como Q, si se definen 2 el 1ero aplica para P y el 2do para Q

<sup>(2)</sup> Se supone que cada entrada positiva es 1

<sup>(3)</sup> En libmf su ausencia indica que no se llevará acabo. Por ejemplo, si no aparece "--nmf" en el llamado de mf-train, implica que las matrices P y Q pueden tener elementos negativos

In [27]:
# A continuación se corre un demo que muestra un poco el uso de estas funciones
# resalta que para programar este demo se definieron unas clases que a su vez
# ocupan las clases principales antes mencionadas de python-limbf
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA python-libmf
%cd /home/miuser/python-libmf/tests
!pwd
!python3 mf_tests.py

/home/miuser/python-libmf/tests
/home/miuser/python-libmf/tests
/home/miuser/.local/lib/python3.6/site-packages/libmf/__init__.py
Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so
iter      tr_rmse          obj
   0       0.5063   3.1186e+02
   1       0.4926   2.9864e+02
   2       0.4550   2.6692e+02
   3       0.3928   2.1779e+02
   4       0.3375   1.7813e+02
   5       0.3037   1.5542e+02
   6       0.2883   1.4492e+02
   7       0.2811   1.3929e+02
   8       0.2779   1.3590e+02
   9       0.2761   1.3346e+02
  10       0.2751   1.3181e+02
  11       0.2752   1.3125e+02
  12       0.2745   1.3031e+02
  13       0.2734   1.2915e+02
  14       0.2733   1.2862e+02
  15       0.2744   1.2903e+02
  16       0.2739   1.2847e+02
  17       0.2731   1.2774e+02
  18       0.2732   1.2758e+02
  19       0.2732   1.2744e+02
testing cross val method
/home/miuser/.local/lib/python3.6/site-

In [28]:
#se tiene una función para generar matrices ralas compactas sin embargo
#se observa que puede arrojar más de un valor para un mismo
#elemento de la matriz rala, vease abajo el renglón [0,1]
#tiene 3 valores asignados (0.166,0.217,0.556).

#El 1er parámetro señala indica que se seleccionará un valor
#de entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar valores entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar 

#Fijaremos una semilla para reproducibilidad
np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)
print('Matriz generada con mf.generate_test_data(20,5,25):\n')
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
print('Tamaño MC:\n',MC.shape)

Matriz generada con mf.generate_test_data(20,5,25):

 [ 0.     1.     0.166]
 [ 19.      4.      0.306]
 [ 14.      3.      0.147]
 [ 18.      4.      0.223]
 [ 18.     1.     0.54]
 [ 15.      3.      0.749]
 [ 2.     2.     0.428]
 [ 11.      3.      0.105]
 [ 9.     3.     0.753]
 [ 8.     5.     0.598]
 [ 13.      0.      0.487]
 [ 8.     4.     0.975]
 [ 9.     2.     0.802]
 [ 15.      3.      0.333]
 [ 7.     5.     0.549]
 [ 4.     5.     0.312]
 [ 16.      4.      0.062]
 [ 6.     1.     0.903]
 [ 18.      0.      0.861]
 [ 3.    3.    0.63]
 [ 0.     1.     0.217]
 [ 0.     1.     0.556]
 [ 16.      5.      0.109]
 [ 6.     5.     0.551]
 [ 7.     0.     0.581]
Tamaño MC:
 (25, 3)


In [29]:
#Defínase un objeto de tipo MF para obtener la factorización deseada
mf_netflix=mf.MF()

#veamos algunas de las propiedades del objeto mf_netflix
print('Tipo de objeto: ',type(mf_netflix))
print('Instance variables: ',mf_netflix.__dict__)

Tipo de objeto:  <class 'libmf.mf.MF'>
Instance variables:  {'model': None, '_options': <libmf.mf.MFParam object at 0x7f73e6d0f6a8>, 'i': None, 'j': None}


In [30]:
#Hasta el momento no se han definido los parámetros para correr el
#modelo de factorización, sin embargo por default se asignan algunos
#valores que a continuación se muestran:
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=8
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [31]:
#Observa que entre los atributos están los parámetros para que corra libmf
#y que definen el modelo (son los últimos 13; los que no les antecede y finalizas
#con "__")
print('Atributos: ', dir(mf_netflix._options))

Atributos:  ['__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'copy_data', 'do_nmf', 'eta', 'fun', 'k', 'lambda_p1', 'lambda_p2', 'lambda_q1', 'lambda_q2', 'nr_bins', 'nr_iters', 'nr_threads', 'quiet']


In [32]:
#Sin embargo podemos definir parámetros distintos pasando los argumentos
#Aquí lo hacemos en forma de kwargs, es decir, con el nombre de las variables
mf_netflix=mf.MF(fun=0,k=10,nr_threads=12,nr_bins=26,nr_iters=20,lambda_p1=0.04,lambda_p2=0.0,lambda_q1=0.04,lambda_q2=0.0,eta=0.1,do_nmf=False,quiet=False,copy_data=True)
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=10
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [33]:
#Si optas por usar los parámetros por default excepto algunos
#puedes pasar los parámetros que quieres definir diferente
mf_netflix=mf.MF(eta=0.2,nr_iters=30)
#Nota que en este ejemplo como no definiste k=10, se inicializa con k=8
#pues es el valor por default
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=8
nr_threads=12
nr_bins=26
nr_iters=30
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.2000
do_nmf= False
quiet= False
copy_data= True


In [34]:
#Generamos una matriz rala al azar y la dividimos en entrenamiento y prueba
#para probar las funciones de factorización
np.random.seed(104)
MC=crear_matriz_rala_compacta(20,5,0.40)
print('El tamaño de MC es:',MC.shape)
MC_valida, MC_entrena=muestra_val_ent(pd.DataFrame(MC,columns=['usuario_id', 'peli_id', 'calif']),0.3)
MC_valida=MC_valida.to_numpy()
print('El tamaño de MC_valida es:',MC_valida.shape)
print('La matriz MC_valida queda así:\n',MC_valida)
MC_entrena=MC_entrena.to_numpy()
print('El tamaño de MC_entrena es:',MC_entrena.shape)

#Implementamos modelo con MC_entrena
mf_netflix.fit(MC_entrena)

El tamaño de MC es: (40, 3)
El tamaño de MC_valida es: (5, 3)
La matriz MC_valida queda así:
 [[ 1  0  5]
 [17  0  3]
 [ 9  0  5]
 [ 6  0  3]
 [ 9  4  2]]
El tamaño de MC_entrena es: (35, 3)


In [35]:
#Para hacer la predicción se le pasa sólo los
#índices de los elementos a predecir, es decir
#una base con sólo 2 columnas que indican el 
#numero de renglon y columnas
pred=mf_netflix.predict(MC_valida[:,0:2])
pred

array([ 6.851,  4.714,  6.869,  4.714,  5.244], dtype=float32)

In [36]:
print('Los valores verdaderos de MC_valida son:',MC_valida[:,2])

Los valores verdaderos de MC_valida son: [5 3 5 3 2]


In [37]:
# #Veamos cómo quedó el modelo
# dir(mf_netflix.model.Q)
# mf_netflix.model.Q.contents

In [38]:
Q=mf_netflix.q_factors()
Q

array([[ 1.155e+00,  8.738e-01,  1.368e+00,  1.894e+00,  1.603e-01,
         3.762e-01,  2.001e-01,  4.839e-01],
       [-0.000e+00, -4.409e-02, -5.840e-01,  4.032e-01,  4.771e-03,
         8.447e-01,  8.528e-01,  9.662e-01],
       [ 2.006e-01, -9.125e-02,  3.898e-01,  7.268e-01, -6.092e-04,
         9.708e-01,  4.906e-01,  5.023e-01],
       [ 1.634e-01,  3.821e-01,  5.503e-02,  1.925e-01,  2.867e-01,
         1.260e-01,  4.063e-01,  5.989e-01],
       [ 2.041e-01,  3.775e-01,  6.300e-01,  3.597e-01,  6.042e-01,
         3.705e-01,  9.813e-01,  1.048e+00],
       [ 4.786e-01,  1.082e+00,  7.867e-01,  4.555e-01,  1.191e+00,
         5.027e-01,  8.306e-01,  1.091e+00],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan],
       [ 1.778e+00,  1.381e+00,  2.924e-01,  8.586e-01,  7.010e-01,
         1.124e+00,  1.390e+00,  1.538e+00],
       [ 2.037e+00,  1.486e+00,  2.222e+00,  1.314e+00,  2.022e+00,
         1.334e+00,  1.586e+

In [39]:
P=mf_netflix.p_factors()
P

array([[ 0.606,  0.654,  0.479,  0.755,  0.656,  0.522,  0.362,  0.751],
       [ 0.765,  0.036,  1.208,  1.361,  1.108,  2.022,  1.775,  0.853],
       [ 0.216,  0.718,  0.659,  0.758,  0.51 ,  0.324,  0.615,  0.865],
       [ 0.   ,  0.787, -0.134, -0.191,  1.012, -0.034,  1.124,  1.051],
       [ 1.657,  1.334,  2.06 ,  0.759,  1.819,  0.654,  1.017,  0.611],
       [ 1.283,  0.498,  0.145,  1.06 ,  0.914,  1.609,  1.306,  1.232],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [ 0.626, -0.   ,  0.536,  0.512,  0.357,  1.154,  0.876,  1.174],
       [ 1.746,  1.303,  0.262,  0.936,  0.716,  1.167,  1.462,  1.522],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    

In [40]:
R=P@np.transpose(Q)
R

array([[  4.095,   1.474,   1.859,   1.372,   2.677,   3.883,     nan,
          5.474,   7.841,   6.881,   2.808,     nan,     nan,   1.278,
            nan,   4.042,   3.045,   5.98 ],
       [  6.851,   3.893,   4.871,   2.272,   5.475,   6.716,     nan,
          9.76 ,  14.95 ,  16.904,   5.7  ,     nan,     nan,   2.045,
            nan,   8.747,   6.394,   9.865],
       [  3.959,   1.525,   1.836,   1.446,   2.94 ,   3.967,     nan,
          5.125,   7.534,   6.712,   3.031,     nan,     nan,   1.177,
            nan,   3.722,   3.3  ,   5.754],
       [  1.026,   1.917,   0.783,   1.629,   2.948,   3.928,     nan,
          4.734,   5.777,   4.592,   3.046,     nan,     nan,   0.969,
            nan,   2.999,   3.922,   3.035],
       [  8.373,   1.063,   3.005,   2.423,   5.393,   8.21 ,     nan,
         10.407,  17.894,  14.695,   6.645,     nan,     nan,   2.686,
            nan,   7.728,   5.77 ,  11.688],
       [  5.733,   3.989,   3.86 ,   2.346,   4.644,   6.076,    

In [41]:
#Podemos ver cómo se ve la matriz R en forma compacta
compactar_matriz_rala(R)
pd.DataFrame(compactar_matriz_rala(R))

,0,1,2
0,0.0,0.0,4.094992
1,0.0,1.0,1.474386
2,0.0,2.0,1.858589
3,0.0,3.0,1.371552
4,0.0,4.0,2.676564
...,...,...,...
149,18.0,10.0,3.949518
150,18.0,13.0,1.428563
151,18.0,15.0,4.120008
152,18.0,16.0,3.609870


In [42]:
#Notese que R justo contiene algunas de las estimaciones arrojadas por
#el método predict, pero no todas
comparativo=pd.DataFrame(columns=['ren','col','calif','R_calif_est','mf_netflix_predict'])
for i in range(MC_valida.shape[0]):
    comparativo.loc[i] = [MC_valida[i,0], MC_valida[i,1], MC_valida[i,2],R[MC_valida[i,0],MC_valida[i,1]],pred[i]]
display(comparativo)

#Observa que R no contiene todas las estimaciones que tiene predict
#checamos unos casos particulares: 4, 9, 11, 12
#nota que todos esos usuarios aparecen tanto en MC_entrena como 
#en MC_valida, y sin embargo la matriz R no tiene las calificaciones
#ni del usuario 11 ni del 12
df_MC_valida=pd.DataFrame(MC_valida,columns=['ren','col','calif'])
df_MC_entrena=pd.DataFrame(MC_entrena,columns=['ren','col','calif'])
print('Usuarios 4, 9, 11 y 12 en MC_valida:')
display(df_MC_valida[df_MC_valida.ren.isin([4,9,11,12])])
print('Usuarios 4, 9, 11 y 12 en MC_entrena:')
display(df_MC_entrena[df_MC_entrena.ren.isin([4,9,11,12])])


#Obsérvese las películas que los usuarios 4,9,11 y 12 vieron
#notamos que son: 0,1,2,4 (de un universo de 5 películas sólo falto la 3)
#En la base de entrena, hubo usuarios que calificaron las películas 0,1,2 y 4
#sin embargo en la de valida, no hubo usuarios que calificaran las películas
#1 y 4, ni la 3
print('Películas MC_valida:')
display(df_MC_valida[df_MC_valida.col.isin([0,1,2,4])])
print('MC_entrena:')
display(df_MC_entrena[df_MC_entrena.col.isin([0,1,2,4])])


,ren,col,calif,R_calif_est,mf_netflix_predict
0,1.0,0.0,5.0,6.850984,6.850984
1,17.0,0.0,3.0,NaN,4.714286
2,9.0,0.0,5.0,6.869184,6.869184
3,6.0,0.0,3.0,NaN,4.714286
4,9.0,4.0,2.0,5.244169,5.244169


Usuarios 4, 9, 11 y 12 en MC_valida:


,ren,col,calif
2,9,0,5
4,9,4,2


Usuarios 4, 9, 11 y 12 en MC_entrena:


,ren,col,calif
2,11,3,4
5,4,4,5
8,4,3,2
23,12,3,4
24,9,2,2
26,12,2,1
28,9,3,5
30,4,0,5


Películas MC_valida:


,ren,col,calif
0,1,0,5
1,17,0,3
2,9,0,5
3,6,0,3
4,9,4,2


MC_entrena:


,ren,col,calif
0,1,1,2
1,2,1,2
3,3,1,1
5,4,4,5
7,10,1,4
9,5,2,5
10,15,1,4
12,14,2,1
14,16,1,3
15,8,2,1


In [43]:
# #Esta paquetería tiene unas clases definidas y métodos con los 
# #cuales corre algunos de los algoritmos

# #Uno de los métodos utilizados para obtener un modelo de recomendación
# #es encontrar 2 matrices cuya multiplicación proporciona la estimación
# #de calificación. Ambas matrices contienen los llamados "factores latentes"
# #que se les puede dar una interpretación de características/gustos
# #Esta factorización puede quedar como P*Q_transpuesta

# mf_engine=mf.MF()
# type(mf_engine)

In [44]:
# #fit method
# mf_engine.fit(MC)
# mf_engine

In [45]:
# #testing factor-p
# P=mf_engine.p_factors()
# print('Matriz p:\n',P)
# print('Tamaño p:\n',P.shape)

In [46]:
# #testing factor-q
# Q=mf_engine.q_factors()
# print('Matriz Q:\n',Q)
# print('Tamaño Q:\n',Q.shape)

In [47]:
# #Se exhibe la multiplicación de matrices P y Q
# R_est=P@np.transpose(Q)
# print('Matriz p:\n',R_est)
# print('Tamaño p:\n',R_est.shape)

In [48]:
# #validación cruzada
# mf_engine.mf_cross_validation(MC)

# Obtención de muestra base netflix

In [49]:
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA DE LAS BASES
#movies_titles_fix.csv y dat_muestra_nflix.csv
%cd /home/miuser/Codigo
#Se guarda una variable con una muestra de las base de netflix ("movies_title_fix.csv")
#A continuación se presentan las características de las películas, con el objeto
#de conocer un poco más los datos utilizados (pero esta no será la base usada para el modelo)
pelis_nombres=pd.read_csv('movies_title_fix.csv',names=["peli_id", "año", "nombre"])
pelis_nombres.head(10) #para ver cómo se encuentran los datos

/home/miuser/Codigo


,peli_id,año,nombre
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [50]:
#La base tiene peli_id, usuario_id_orig, calif, fecha y usuario_id. Se trabajará solo
#con las columnas de usuario, pelicula y calificación
df_muestra_netflix=pd.read_csv('dat_muestra_nflix.csv')
df_muestra_netflix_libmf=df_muestra_netflix[['usuario_id', 'peli_id', 'calif']]
df_muestra_netflix_libmf.shape

(20968941, 3)

In [51]:
#Se genera una base de validación que contendrá el 20% de los datos
#y una base de entrenamiento que tendrá el restante 80%
random.seed(28882)
data_valida, data_entrena=muestra_val_ent(df_muestra_netflix_libmf,0.2)

#puedes guardar las bases como un .csv dentro del directorio donde está este notebook
data_valida.to_csv(r'data_valida.csv', index = False)    
data_entrena.to_csv(r'data_entrena.csv', index = False)    

# #Base de todos los usuarios
# usuarios=pd.DataFrame(df_muestra_netflix_libmf['usuario_id'].unique(), columns=['usuario_id'])
# #Muestreo de usuarios
# valida_usuarios=usuarios.sample(frac=0.2)  

# #Base de todas las películas
# peliculas=pd.DataFrame(df_muestra_netflix_libmf['peli_id'].unique(), columns=['peli_id'])
# #Muestreo de películas
# valida_pelis=peliculas.sample(frac=0.2)

In [52]:
# #df_valida tiene las columnas usuario_id, peli_id y calif, pero
# #sólo incluye los usuarios muestreados en valida_usuarios
# df_valida=pd.merge(df_muestra_netflix_libmf,valida_usuarios, on ='usuario_id', how='inner')
# #data_valida filtra de la base df_valida los pelis_id muestreas
# #en valida_pelis
# data_valida=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
# #guardamos data_valida como un .csv dentro del directorio donde está este notebook
# data_valida.to_csv(r'muestra_valida.csv', index = False)
# left= pd.merge(df_muestra_netflix_libmf, data_valida, how='left',on=['usuario_id', 'peli_id'])
# data_entrena=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]

In [53]:
display(data_valida.head(5))
data_valida.shape

,usuario_id,peli_id,calif
0,4,128,4
1,89,128,5
2,125,128,2
3,452,128,1
4,642,128,2


(877113, 3)

In [54]:
display(data_entrena.head(5))
data_entrena.shape

,usuario_id,peli_id,calif_x
0,1,1,3
1,2,1,3
2,3,1,4
3,4,1,4
4,5,1,4


(20091828, 3)

In [55]:
#podemos cambiar el directorio para que se guarde la base
#dentro del directorio donde están las funciones de libmf
#se guarda con la extensión .tr.txt porque es la extensión
#que requiere la paquetería de libmf
%cd /home/miuser/libmf/demo
np.savetxt('data_valida.te.txt', data_valida.to_numpy(), delimiter=' ', fmt="%.0f")
np.savetxt('data_entrena.tr.txt', data_entrena.to_numpy(), delimiter=' ', fmt="%.0f")

/home/miuser/libmf/demo


In [56]:
#La base de df_muestra_valida_libmf ha quedado dividida por
#dos dataframes exluyenntes: data_valida y data_entrena
print("La suma de elementos de data_valida + data_entrena:",data_valida.shape[0] + data_entrena.shape[0])
print("El total de elementos de df_muestra_netflix_libmf es:",df_muestra_netflix_libmf.shape[0])

La suma de elementos de data_valida + data_entrena: 20968941
El total de elementos de df_muestra_netflix_libmf es: 20968941


In [57]:
modelo_base_ref(data_valida.to_numpy())

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
4,4,128,4.207792,2.954545,3.609339
4,4,187,4.207792,3.078089,3.609339
4,4,571,4.207792,3.953481,3.609339
4,4,586,4.207792,3.247706,3.609339
4,4,708,4.207792,3.371808,3.609339
...,...,...,...,...,...
99912,99912,17307,5.000000,4.521127,3.609339
99928,99928,16169,2.000000,3.131050,3.609339
99937,99937,17293,3.000000,2.980172,3.609339


# Librería libmf

In [58]:
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA libmf
#Prueba correr el demo que viene en el la librería libmf
%cd /home/miuser/libmf/demo
!pwd
!bash demo.sh

/home/miuser/libmf/demo
/home/miuser/libmf/demo
--------------------------------
Real-valued matrix factorization
--------------------------------
iter      tr_rmse      va_rmse          obj
   0       2.4800       1.3263   3.2496e+04
   1       1.3026       1.0788   1.0211e+04
   2       0.8889       1.0374   5.6777e+03
   3       0.8030       1.0216   4.9486e+03
   4       0.7484       1.0110   4.5383e+03
   5       0.7002       1.0127   4.1921e+03
   6       0.6540       1.0100   3.9099e+03
   7       0.6092       1.0082   3.6359e+03
   8       0.5596       1.0096   3.3746e+03
   9       0.5109       1.0106   3.1490e+03
MAE = 0.7841
---------------------------
binary matrix factorization
---------------------------
iter   tr_logloss   va_logloss          obj
   0       0.6878       0.8163   3.4202e+03
   1       0.6967       0.8155   3.4628e+03
   2       0.6816       0.8186   3.3888e+03
   3       0.6668       0.8117   3.3180e+03
   4       0.6509       0.8018   3.2434e+03
   5    

In [59]:
%cd /home/miuser/libmf/demo

/home/miuser/libmf/demo


In [60]:
#train a model using the default parameters
!/home/miuser/libmf/mf-train real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9774   2.2897e+04
   1       0.9888   8.1391e+03
   2       0.8673   6.9456e+03
   3       0.8169   6.5041e+03
   4       0.7855   6.2004e+03
   5       0.7663   6.0164e+03
   6       0.7523   5.8733e+03
   7       0.7420   5.8016e+03
   8       0.7314   5.6968e+03
   9       0.7242   5.6156e+03
  10       0.7152   5.5484e+03
  11       0.7116   5.5164e+03
  12       0.7051   5.4582e+03
  13       0.7006   5.3974e+03
  14       0.6965   5.3933e+03
  15       0.6921   5.3626e+03
  16       0.6852   5.2838e+03
  17       0.6774   5.2383e+03
  18       0.6777   5.2374e+03
  19       0.6733   5.2045e+03


In [61]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0.05
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.01
!/home/miuser/libmf/mf-train -l1 0.05 -l2 0.01 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9886   2.2885e+04
   1       0.9930   8.0382e+03
   2       0.8612   6.8147e+03
   3       0.8053   6.3506e+03
   4       0.7699   6.0599e+03
   5       0.7444   5.8568e+03
   6       0.7264   5.7192e+03
   7       0.7099   5.5972e+03
   8       0.6946   5.4801e+03
   9       0.6802   5.3849e+03
  10       0.6646   5.2691e+03
  11       0.6518   5.1767e+03
  12       0.6394   5.0996e+03
  13       0.6257   5.0172e+03
  14       0.6100   4.9104e+03
  15       0.5997   4.8493e+03
  16       0.5869   4.7715e+03
  17       0.5751   4.6978e+03
  18       0.5627   4.6223e+03
  19       0.5519   4.5683e+03


In [62]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.03
!/home/miuser/libmf/mf-train -l1 0.015,0 -l2 0.01,0.005 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9589   1.9664e+04
   1       0.9769   5.2544e+03
   2       0.8479   4.0808e+03
   3       0.7921   3.6291e+03
   4       0.7640   3.4107e+03
   5       0.7346   3.1928e+03
   6       0.7212   3.0957e+03
   7       0.7071   2.9967e+03
   8       0.6928   2.9004e+03
   9       0.6788   2.8049e+03
  10       0.6648   2.7127e+03
  11       0.6518   2.6294e+03
  12       0.6380   2.5424e+03
  13       0.6235   2.4527e+03
  14       0.6106   2.3749e+03
  15       0.5960   2.2895e+03
  16       0.5833   2.2165e+03
  17       0.5705   2.1454e+03
  18       0.5573   2.0725e+03
  19       0.5442   2.0040e+03


In [63]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.03
!/home/miuser/libmf/mf-train -l1 0.015,0 -l2 0.01,0.005 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9579   1.9645e+04
   1       0.9769   5.2544e+03
   2       0.8479   4.0808e+03
   3       0.7937   3.6409e+03
   4       0.7606   3.3850e+03
   5       0.7409   3.2370e+03
   6       0.7226   3.1082e+03
   7       0.7070   2.9966e+03
   8       0.6916   2.8917e+03
   9       0.6787   2.8038e+03
  10       0.6651   2.7150e+03
  11       0.6520   2.6308e+03
  12       0.6379   2.5413e+03
  13       0.6245   2.4593e+03
  14       0.6096   2.3687e+03
  15       0.5977   2.2996e+03
  16       0.5838   2.2204e+03
  17       0.5676   2.1276e+03
  18       0.5577   2.0751e+03
  19       0.5445   2.0037e+03


In [64]:
#train a BMF model using logarithmic loss and the following parameters:
#coefficient of L1-norm regularization on P = 0
#coefficient of L1-norm regularization on Q = 0.01
#latent factors = 100
#iterations = 30
#learning rate = 0.02
#threads = 4
!/home/miuser/libmf/mf-train -f 5 -l1 0,0.02 -k 100 -t 30 -r 0.02 -s 4 binary_matrix.tr.txt model

iter   tr_logloss          obj
   0       0.6800   4.0074e+03
   1       0.6950   4.0132e+03
   2       0.6915   3.9387e+03
   3       0.6891   3.8787e+03
   4       0.6873   3.8318e+03
   5       0.6859   3.7932e+03
   6       0.6848   3.7617e+03
   7       0.6840   3.7350e+03
   8       0.6833   3.7123e+03
   9       0.6828   3.6930e+03
  10       0.6824   3.6764e+03
  11       0.6820   3.6617e+03
  12       0.6817   3.6491e+03
  13       0.6815   3.6379e+03
  14       0.6813   3.6278e+03
  15       0.6811   3.6190e+03
  16       0.6810   3.6106e+03
  17       0.6808   3.6031e+03
  18       0.6807   3.5966e+03
  19       0.6806   3.5904e+03
  20       0.6805   3.5848e+03
  21       0.6805   3.5796e+03
  22       0.6804   3.5748e+03
  23       0.6803   3.5703e+03
  24       0.6803   3.5662e+03
  25       0.6802   3.5624e+03
  26       0.6802   3.5587e+03
  27       0.6802   3.5554e+03
  28       0.6801   3.5523e+03
  29       0.6801   3.5493e+03


In [65]:
#use real_matrix.te.txt for hold-out validation
!/home/miuser/libmf/mf-train -p real_matrix.te.txt real_matrix.tr.txt model

iter      tr_rmse      va_rmse          obj
   0       1.9774       1.1544   2.2897e+04
   1       0.9888       1.0483   8.1391e+03
   2       0.8673       1.0275   6.9456e+03
   3       0.8128       1.0148   6.4561e+03
   4       0.7861       1.0110   6.1974e+03
   5       0.7661       1.0083   5.9971e+03
   6       0.7529       1.0055   5.9000e+03
   7       0.7369       1.0052   5.7465e+03
   8       0.7317       1.0042   5.6978e+03
   9       0.7239       1.0047   5.6207e+03
  10       0.7170       1.0036   5.5766e+03
  11       0.7102       1.0055   5.4818e+03
  12       0.7051       1.0041   5.4557e+03
  13       0.6997       1.0077   5.4035e+03
  14       0.6961       1.0044   5.3847e+03
  15       0.6889       1.0028   5.3339e+03
  16       0.6851       1.0072   5.2869e+03
  17       0.6811       1.0070   5.2628e+03
  18       0.6778       1.0066   5.2348e+03
  19       0.6716       1.0087   5.1874e+03


In [66]:
#do five fold cross validation
!/home/miuser/libmf/mf-train -v 5 real_matrix.tr.txt

fold      rmse
   0    1.4035
   1    1.4229
   2    1.5077
   3    1.5462
   4    1.5085
 avg    1.4778


In [67]:
#do non-negative matrix factorization with generalized KL-divergence
!/home/miuser/libmf/mf-train -f 2 --nmf real_matrix.tr.txt

iter       tr_gkl          obj
   0       1.0060   8.0920e+03
   1       0.2079   3.8743e+03
   2       0.1766   3.6619e+03
   3       0.1584   3.5389e+03
   4       0.1448   3.4499e+03
   5       0.1368   3.3777e+03
   6       0.1323   3.3341e+03
   7       0.1276   3.3025e+03
   8       0.1239   3.2633e+03
   9       0.1206   3.2269e+03
  10       0.1190   3.2150e+03
  11       0.1166   3.2040e+03
  12       0.1140   3.1681e+03
  13       0.1132   3.1699e+03
  14       0.1116   3.1533e+03
  15       0.1099   3.1390e+03
  16       0.1092   3.1320e+03
  17       0.1077   3.1253e+03
  18       0.1071   3.1210e+03
  19       0.1063   3.1150e+03


In [68]:
#do not print message to screen
!/home/miuser/libmf/mf-train --quiet real_matrix.tr.txt

In [69]:
#do disk-level training
!/home/miuser/libmf/mf-train --disk real_matrix.tr.txt

iter      tr_rmse          obj
   0       1.5804   1.5639e+04
   1       1.1927   1.0373e+04
   2       0.8689   6.9306e+03
   3       0.8087   6.4423e+03
   4       0.7716   6.0912e+03
   5       0.7579   5.9491e+03
   6       0.7427   5.8056e+03
   7       0.7293   5.6905e+03
   8       0.7222   5.6206e+03
   9       0.7131   5.5715e+03
  10       0.7109   5.5121e+03
  11       0.7059   5.4805e+03
  12       0.6988   5.4079e+03
  13       0.6901   5.3604e+03
  14       0.6880   5.3041e+03
  15       0.6850   5.2953e+03
  16       0.6796   5.2436e+03
  17       0.6749   5.2191e+03
  18       0.6723   5.2008e+03
  19       0.6657   5.1567e+03


In [70]:
#do prediction
!/home/miuser/libmf/mf-predict real_matrix.te.txt model output

RMSE = 1.0087


In [71]:
#do prediction and output MAE
!/home/miuser/libmf/mf-predict -e 1 real_matrix.te.txt model output

MAE = 0.7857


In [72]:
#Pruebese los diferentes modelos ejemplo con data_entrena y data_valida
#En general estos algoritmos corren muy rápido
!/home/miuser/libmf/mf-train data_entrena.tr.txt model

iter      tr_rmse          obj
   0       0.9819   3.0143e+07
   1       0.9365   2.8337e+07
   2       0.9305   2.8106e+07
   3       0.9270   2.8001e+07
   4       0.9219   2.7863e+07
   5       0.9149   2.7675e+07
   6       0.9077   2.7479e+07
   7       0.9021   2.7323e+07
   8       0.8982   2.7207e+07
   9       0.8955   2.7128e+07
  10       0.8936   2.7075e+07
  11       0.8922   2.7033e+07
  12       0.8911   2.7004e+07
  13       0.8902   2.6975e+07
  14       0.8895   2.6955e+07
  15       0.8889   2.6935e+07
  16       0.8884   2.6925e+07
  17       0.8879   2.6911e+07
  18       0.8874   2.6898e+07
  19       0.8869   2.6881e+07
